# Fake News Detection - BERT Model

This notebook implements a BERT-based model for fake news detection.

In [ ]:
import pandas as pd
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

# Load preprocessed data
df = pd.read_csv("../data/preprocessed_data.csv")

# Sample data for quick training
df_small = df.sample(5000, random_state=42)
print("Using sample of size:", len(df_small))

In [ ]:
# Prepare train/val split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_small["content"].tolist(),
    df_small["label"].tolist(),
    test_size=0.2
)

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)

In [ ]:
# Create datasets
train_dataset = Dataset.from_dict({
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels
})

val_dataset = Dataset.from_dict({
    'input_ids': val_encodings['input_ids'],
    'attention_mask': val_encodings['attention_mask'],
    'labels': val_labels
})

In [ ]:
# Initialize model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

# Set up training arguments
training_args = TrainingArguments(
    output_dir='../results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    evaluation_strategy="epoch"
)

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [ ]:
# Train the model
trainer.train()

In [ ]:
# Evaluate
from sklearn.metrics import classification_report

preds_output = trainer.predict(val_dataset)
preds = preds_output.predictions.argmax(axis=1)
print(classification_report(val_labels, preds))